# Intel 仅权重量化
## 使用 Intel 扩展进行 Transformer 流水线任务的 Huggingface 模型仅权重量化

Hugging Face 模型可以通过 `WeightOnlyQuantPipeline` 类在本地运行，并进行仅权重量化。

[Hugging Face Model Hub](https://huggingface.co/models) 主持了超过 120k 个模型、20k 个数据集和 50k 个演示应用 (Spaces)，所有这些都是开源且公开可用的，在一个在线平台上，人们可以轻松协作和共同构建机器学习。

这些可以通过此本地流水线包装器类从 LangChain 调用。

要使用本功能，您需要安装 ``transformers`` Python [软件包](https://pypi.org/project/transformers/)，以及 [pytorch](https://pytorch.org/get-started/locally/) 和 [intel-extension-for-transformers](https://github.com/intel/intel-extension-for-transformers)。

In [ ]:
%pip install transformers --quiet
%pip install intel-extension-for-transformers

### 模型加载

可以通过 `from_model_id` 方法指定模型参数来加载模型。模型参数包括 `intel_extension_for_transformers` 中的 `WeightOnlyQuantConfig` 类。

In [ ]:
from intel_extension_for_transformers.transformers import WeightOnlyQuantConfig
from langchain_community.llms.weight_only_quantization import WeightOnlyQuantPipeline

conf = WeightOnlyQuantConfig(weight_dtype="nf4")
hf = WeightOnlyQuantPipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    quantization_config=conf,
    pipeline_kwargs={"max_new_tokens": 10},
)

它们也可以通过直接传入一个现有的 `transformers` pipeline 来加载

In [ ]:
from intel_extension_for_transformers.transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, pipeline

model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
pipe = pipeline(
    "text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
)
hf = WeightOnlyQuantPipeline(pipeline=pipe)

### 创建链

模型加载到内存后，您可以将其与提示组合以形成一个链。

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

### CPU 推理

目前 intel-extension-for-transformers 仅支持 CPU 设备推理。稍后将支持 Intel GPU。当在带有 CPU 的机器上运行时，您可以指定 `device="cpu"` 或 `device=-1` 参数将模型置于 CPU 设备上。
CPU 推理的默认值为 `-1`。

In [ ]:
conf = WeightOnlyQuantConfig(weight_dtype="nf4")
llm = WeightOnlyQuantPipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    quantization_config=conf,
    pipeline_kwargs={"max_new_tokens": 10},
)

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

### Batch CPU 推理

你也可以在 CPU 上以批处理模式运行推理。

In [ ]:
conf = WeightOnlyQuantConfig(weight_dtype="nf4")
llm = WeightOnlyQuantPipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    quantization_config=conf,
    pipeline_kwargs={"max_new_tokens": 10},
)

chain = prompt | llm.bind(stop=["\n\n"])

questions = []
for i in range(4):
    questions.append({"question": f"What is the number {i} in french?"})

answers = chain.batch(questions)
for answer in answers:
    print(answer)

### Intel-extension-for-transformers 支持的数据类型

我们支持将权重量化为以下数据类型进行存储（`weight_dtype` in `WeightOnlyQuantConfig`）：

*   **int8**：使用 8 位数据类型。
*   **int4_fullrange**：使用 int4 范围的 -8 值，与正常的 int4 范围 [-7,7] 相比。
*   **int4_clip**：将值裁剪并保留在 int4 范围内，其余设置为零。
*   **nf4**：使用归一化浮点 4 位数据类型。
*   **fp4_e2m1**：使用常规浮点 4 位数据类型。“e2”表示使用 2 位表示指数，“m1”表示使用 1 位表示尾数。

虽然这些技术将权重存储为 4 位或 8 位，但计算仍然以 float32、bfloat16 或 int8 进行（`compute_dtype` in `WeightOnlyQuantConfig`）：
*   **fp32**：使用 float32 数据类型进行计算。
*   **bf16**：使用 bfloat16 数据类型进行计算。
*   **int8**：使用 8 位数据类型进行计算。

### 支持的算法矩阵

Intel-extension-for-transformers 中支持的量化算法（`algorithm` in `WeightOnlyQuantConfig`）：

| 算法   | PyTorch | LLM Runtime |
| :----- | :------ | :---------- |
| RTN    | &#10004; | &#10004;    |
| AWQ    | &#10004; | 敬请期待    |
| TEQ    | &#10004; | 敬请期待    |
> **RTN:** 一种我们可以直观理解的量化方法。它不需要额外的训练数据集，是一种非常快速的量化方法。一般来说，RTN 会将权重转换为均匀分布的整数数据类型，但有些算法（例如 Qlora）提出了非均匀 NF4 数据类型，并证明了其理论上的最优性。

> **AWQ:** 证明了仅保护 1% 的显著权重就可以大大减少量化误差。显著权重通道是通过观察每个通道的激活和权重的分布来选择的。显著权重在量化前乘以一个较大的缩放因子以进行保留，然后进行量化。

> **TEQ:** 一种可训练的等价变换，可在仅权重量化中保持 FP32 精度。它受到 AWQ 的启发，同时提供了一种新的解决方案来搜索激活和权重之间最优的每通道缩放因子。